In [ ]:
from mip import Model, xsum, minimize, BINARY

alpha = 1
beta = 0
qmin = 1/2
qmax = 1
Q = 1/4

m = Model()

theta = [m.add_var(var_type=BINARY)]
q = [m.add_var()]

m.objective = minimize(alpha * q[0] + beta - beta * (1 - theta[0]))

#Constraints:
m += q[0] <= qmax * theta[0]
m += q[0] >= qmin * theta[0]
m += q[0] == Q

m.optimize()

q[0].x, theta[0].x